# devlog 2023-12-05

_author: Tyler Coles_

A brief tour of changes to epymorph due to the refactor effort.

## Reorganization

A lot of code changed location, hopefully to a more sensible structure. Here's a listing of the main epymorph packages:

- **cli**: all of the CLI command implementation moved here. argparse definitions are now located next to their handler implementations.
- **data**: (didn't change much) this is still epymorph's built-in model implementations
- **engine**: simulation execution happens here; basically anything that _requires_ the combination of GEO/IPM/MM in order to work
- **geo**: (didn't change much) the geo and ADRIO logic lives here
- **movement**: movement functionality (except for movement execution) moved here -- movement specs, parsing, compilation

There's no IPM package as of yet -- the analog to the `epymorph.movement` is the `epymorph.compartment_model` module.

## Imports

We now provide a curated set of exports for easy importing in user code. Users can now `from epymorph import *` to get most of the things  they'll need to run basic epymorph simulations. Similarly, they can `from epymorph.compartment_model import *` to get most of the things needed to build custom IPMs. We may do more of this in the future.

## Simulation API

A lot of details around simulation setup and execution changed, but sometimes only slightly. Instead of a Simulation, you'll construct a StandardSimulation. StandardSimulation asks you to provide a complete configuration in one go, rather than breaking it up into two steps (construct and run, before). There's a TimeFrame object instead of separate start date and duration in days to describe the length of the simulation. You supply an RNG constructor function instead of an RNG instance. The Output object has been upgraded with more info and restructured a bit. And so on.

`param` changed to `params` to try to be more consistent across the application.

toml file input used to specify the simulation time frame like `duration = "120d"` but now it's `duration_days = 120`.

_BONUS!_ Simulation params will no longer yell at you if they want floats but you give them ints. Ahhh, isn't that nice?

See USAGE examples for more detail on working with the new API.

## Plotting

There's a `epymorph.plots` module now, so you don't have to import from `epymorph.run`. In addition, those plots are more fully featured. They will print more useful labels for geo nodes, IPM compartments, and IPM events. `plot_pop` is no longer log scale by default (but supports log scale as an option).

## Registering models in the library

When adding a GEO, IPM, or MM model to epymorph's built in library using a Python script (most common for IPMs), you used to have to update `epymorph.data.__init__` to include it. Now you can just decorate the model constructor function and epymorph will auto-discover it. (Also it needs to be in the correct `epymorph.data.<model_type>` package.) Example:

```python
from epymorph.data import registry

@registry.ipm('pei')
def load() -> CompartmentModel:
    symbols = create_symbols(...)
    # ... make the rest of the IPM ...
```

## IPM API

You used to specify IPM parameters like this:

```python
symbols = create_symbols(
    compartments=[...],
    attributes=[
        param('D', 'infection_duration', Shapes.TxN, float, allow_broadcast=True),
        param('L', 'immunity_duration', Shapes.TxN, float, allow_broadcast=True),
        geo('H', 'humidity', Shapes.TxN, float, allow_broadcast=True),
    ])
```

But now you should do it like this:

```python
symbols = create_symbols(
    compartments=[...],
    attributes=[
        param('infection_duration', Shapes.TxN, float, 'D'),
        param('immunity_duration', Shapes.TxN, float, 'L'),
        geo('humidity', Shapes.TxN, float, 'H'),
    ])
```

Notice the ordering of function arguments changed to put the symbolic name at the end of the declaration rather than at the start. This is to reflect that we expect it to be used least often.

And `allow_broadcast` is no longer an option -- we just assume everything allows broadcast.